## Práctica de validación/selección de modelos

- [Método Hold-out](#Método-Hold-out)
- [Método validación cruzada de k particiones](#Método-validación-cruzada-de-k-particiones)
- [Método Leave-One-Out](#Método-Leave-One-Out)

Importamos todas las librerías que vamos a utilizar durante la práctica.

In [12]:
# %load ../../standard_import.txt
import pandas as pd
import numpy as np
from test_helper import Test

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)
 
#%config InlineBackend.figure_formats = {'pdf',}
%matplotlib inline

La librería scikit-learn nos ofrece muchas funcionalidades para realizar procesos de aprendizaje automático. En esta práctica vamos a utilizar las funciones que realizan métodos de validación de modelos. Es decir, metodologías que validen la calidad de los modelos y que por tanto, se pueden aplicar para seleccionar el mejor modelo (o los mejores valores de los híper-parámetros de un clasificador) para resolver un problema dado. Todas estas funciones están dentro de la librería [*model_selection*](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection). Para ilustrar el funcionamiento de dichas metodologías, utilizaremos el clasificador de los k vecinos más cercanos.

Para desarrollar la práctica vamos a trabajar con el dataset [*Ionosphere*](https://archive.ics.uci.edu/ml/datasets/Ionosphere). Este dataset contiene la información de un problema en el que un sistema de radar recoge información de 16 antenas con el objetivo de ver si los electrones presentan algún tipo de estructura en la ionosfera (Bueno) o no (Malo).

En el siguiente código se deben leer los datos del problema Ionosphere almacenados en un archivo *ionosphere.csv*. En este caso la variable de salida es la llamada Class.

Realiza la lectura de los datos almacenados en el fichero *ionosphere.csv* y guarda el resultado en una variable llamada ion. A partir de la variable ion, generad las variables X (información de entrada) e y (información de salida). Para eliminar variables (o instancias) de un DataFrame se puede utilizar el método [*drop*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) de Pandas.

In [13]:
# Se realiza la lectura del dataset sonar utilizando pandas
    # La primera línea contiene los nombres de las variables
    # header=None se utilizaría en caso de que no existiera una primera línea con los nombres para las variables

ion = pd.read_csv('ionosphere.csv')

# nombreAtSalida = <RELLENAR>

# Generamos los datos de entrada y de salida: nos quedamos con las columnas correspondientes en cada caso
X = ion.loc[:, ion.columns != 'Class'].copy()
y = ion.loc[:,'Class'].copy()

### Método Hold-out

El método de validación hold-out crea un conjunto de ejemplos de entrenamiento y uno de test a partir del conjunto de datos inicial. Para ello, se determina el porcentaje de ejemplos a utilizar como conjunto de entrenamiento. Estos ejemplos serán escogidos aleatoriamente. El resto de ejemplos (no asignados al conjunto de entrenamiento) serán asignados al conjuntos de test.

Como hemos dicho anteriormente, en la librería *model_selection* de Scikit-learn podemos encontrar una función que realiza este proceso. Por tanto, en primer lugar, para poder utilizarla se debe importar dicha librería de este modo

    from sklearn import model_selection

Dentro de esta librería se encuentra la función llamada [*train_test_split*](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) que nos realiza el proceso de división de los datos aplicando la técnica Hold-out. La función recibe varias variables de entrada y devuelve varias de salida, la llamada a dicha función es la siguiente:

    X_train, X_test, y_train, y_test = model_selection.train_test_split(inputData, outputData, train_size=porcentajeTrain, stratify=variableEstratificacion, random_state=semilla)

* Los parámetros de entrada son:
    * inputData: los datos de entrada
    * outputData: los datos de salida
    * porcentajeTrain: la proporción de ejemplos de entrenamiento
    * semilla: valor de la semilla para poder generar los mismos números aleatorios y poder reproducir los resultados.
    * variableEstratificacion: variable a utilizar para realizar el proceso de estatificación.
* Los parámetros de salida son:
    * X_train: los datos de entrada del conjunto de entrenamiento
    * X_test: los datos de entrada del conjunto de test
    * y_train: los datos de salida del conjunto de entrenamiento
    * y_test: los datos de salida del conjunto de test

Ejercicio: utilizar el método de validación hold-out para obtener el accuracy de entrenamiento y de test con el dataset Ionosphere y utilizando KNN con la configuración por defecto vista en la práctica anterior. Utilizad el 80% de los ejemplos para el conjunto de entrenamiento y el resto para el conjunto de test. Utilizar como semilla el número 12 y recordad utilizar la estratificación.

In [14]:
# Se importan las 3 librerías necesarias para resolver el ejercicio
from sklearn import metrics, model_selection, neighbors

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                train_size=0.8,stratify=y, random_state=123)

# Entrenamiento de KNN con los valores por defecto de los híper-parámetros
knn = neighbors.KNeighborsClassifier()
knn.fit(X_train, y_train)

# Se obtiene el rendimiento en train y test
y_pred_train = knn.predict(X_train)
y_pred_test = knn.predict(X_test)

accTrain = metrics.accuracy_score(y_train,y_pred_train)*100
accTest = metrics.accuracy_score(y_test,y_pred_test)*100

In [15]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
Test.assertEquals(round(accTrain, 2), 86.43, 'Valor de accuracy en train incorrecto')
Test.assertEquals(round(accTest, 2), 83.10, 'Valor de accuracy en test incorrecto')

1 test passed.
1 test passed.


Una vez que tenemos los conjuntos de entrenamiento y de test, podemos seleccionar los mejores valores del o híper-parámetros del algoritmo KNN (o del que estemos utilizando) aplicando esta metodología. Para ello, se deben devidir los datos de entrenamiento en otros dos conjuntos: el de entrenamiento y el de validación. Una vez que los tenemos, el conjunto de entrenamiento será utilizado para realizar el aprendizaje del clasificador con una configuración dada y cuyo rendimiento será evaluado con los ejemplos del conjunto de validación. La mejor configuración del clasificador será aquella que obtenga el mejor rendimiento con el conjunto de validación.

Para evitar realizar este proceso de forma *manual*, Scikit-learn ofrece una clase que nos ayuda a realizar dicho proceso. Esta clase también está dentro del paquete **model_selection** y se llama [*GridSearchCV*](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Su llamada y sus parámetros principales son los siguientes:

    clasificadores = model_selection.GridSearchCV(clasificador, param_grid, scoring=tipoRendimiento, cv=metodoValidacion, return_train_score=devolverResultadosTrain, n_jobs=numeroTareasParalelo, verbose=informacionMostrar)

* clasificador: es el clasificador del que deseamos conocer su mejor configuración. Es decir, la variable en la que almacenamos la llamada en la que definimos el clasificador (sin híper-parámetros de entrada o con los que deseemos fijar).
* param_grid: es un diccionario cuyas claves son los nombres de los híper-parámetros a determinar su mejor configuración. Para cada clave su valor es una lista con los valores que se desean probar para dicho híper-parámetro. Por ejemplo, para el clasificador que vamos a "optimizar" podría ser:
    * KNN: {'n_neighbors': [1, 3, 5, 7, 9], 'weights': ['uniform', 'distance'], 'p': [1, 2, 1.5, 3]}
* tipoRendimiento: determina el tipo de medida a utilizar para evaluar la calidad del clasificador. Si no se especifica se utiliza la que aplica el clasificador a optimizar (normalmente todos utilizar el accuracy). Los posibles valores de este parámetro son:
    * ’accuracy’: accuracy rate
    * ‘precision’: precisión
    * ‘recall’: recall o true positive rate
    * ‘roc_auc’: área bajo la curva ROC
    * Se podrían utilizar otras métricas de rendimiento mediante el uso de [*make_scorer*](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
* metodoValidacion: modelo de validación cruzada a utilizar:
    * Si es un valor entero determina el número de particiones para realizar la validación cruzada de K particiones. Por defecto su valor *None* que implica utilizar 5 como número de particiones.
    * Un objeto de un método de validación de modelos.
* devolverResultadosTrain: variable booleana que determina si se devuelven los resultados de entrenamiento o no. Por defecto su valor es False.
* numeroTareasParalelo: valor entero que determina el número de tareas a realizar en paralelo. Por defecto no se utiliza paralelización. Si se le asigna -1 se utilizan todos los procesadores disponibles.
* informacionMostrar: valor entero que determina la información a mostrar en el proceso. A mayor valor se muestra más información. Por defecto (0) no se muestra nada.

El parámetro de salida es un conjunto de clasificadores (todas las posibles combinaciones de los híper-parámetros a utilizar). Al igual que hacemos con un solo clasificador, debemos entrenar todos ellos con los datos de train (llamada al método *fit* con la variable donde se almacena el conjunto de clasificadores). En este caso, la llamada al entrenamiento ya realiza internamente tanto el entrenamiento como la evaluación del rendimiento de los clasificadores en el conjunto de validación. Por este motivo, tras entrenarlos (llamada a *fit*), ya se ha calculado internamente la mejor configuración. La mejor configuración está almacenada en el campo *best_params_* y su rendimiento asociado está almacenado en el campo *best_score_*. Podemos visualizarlos con *print*:

    print(clasificadores.best_params_)
    print(clasificadores.best_score_)

Además, por defecto (refit es True) se entrena un clasificador con la mejor configuración encontrada que permite utilizar directamente el método *predict* sobre la instancia de *GridSearchCV* (usando dicho clasificador). Este clasificador también lo tenemos disponible mediante el atributo 

    clasificadores.best_estimator_
    
Para comprobar que efectivamente la configuración mostrada es la mejor, podemos visualizar los resultados de todos los clasificadores considerados (todas las combinaciones de híper-parámetros posibles). Para que el siguiente código funcione debéis especificar que se devuelvan los resultados de entrenamiento en el constructor. Para ello se debe utilizar el siguiente código:

    resultadosMostrar = zip(clasificadores.cv_results_['params'],clasificadores.cv_results_['mean_test_score'],clasificadores.cv_results_['mean_train_score'])
    for params, mean_test_score, mean_train_score in resultadosMostrar:
        print("%0.3f (Train: %0.3f) for %r" % (mean_test_score, mean_train_score, params))
        print()

Se puede observar que en la variable *resultadosMostrar* estamos utilizando el atributo *cv_results_* (es un DataFrame de Pandas) y seleccionando algunas de sus columnas para mostrar (podríamos incluir más información).

En este caso, vamos a aplicar Hold-out como metodología de selección de modelos. Para ello, se puede utilizar el método [*StratifiedShuffleSplit*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html) como método de validación dentro de *GridSearchCV* ya que permite que haga una división de ejemplos en entrenamiento y validación utilizando la estratificación. Para ello, se le debe especificar que haga una sola división de los ejemplos (*n_splits*) y que utilice el 20% de los mismos como conjunto de validación (*test_size*).

Ejercicio: determina la mejor configuración de KNN utilizando los valores de los híper-parámetros mostrados en el texto anterior utilizando Hold-out. Una vez se haya obtenido la mejor configuración, se debe obtener el rendimiento de la misma tanto con los ejemplos de entrenamiento como con los de test obtenidos previamente.

NOTA: Los ejemplos asignados a cada partición dentro de *GridSearchCV* se escogen al azar por lo que de ejecución a ejecución los resultados pueden variar. Para evitar este comportamiento podemos determinar la semilla a utilizar a la hora de generar los números aleatorios. La semilla será un número entero que implica que el número aleatorio generado es siempre el mismo. Por tanto, los ejemplos irán siempre a la misma partición y los resultados serán siempre los mismo. Para fijar la semilla de Numpy, que es la utilizada, se utiliza la siguiente instrucción:

    np.random.seed(valorEntero)

In [16]:
# Se fija la semilla de numpy para que la generación aleatoria siempre nos de los mismos números
np.random.seed(seed=12)
# Realizamos el proceso para KNN por lo que hay que llamar al constructor de dicho clasificador
knn = neighbors.KNeighborsClassifier()
# Se define el grid de parámetros a utilizar
    # Estos parámetros nos darán todas las posibles configuraciones del clasificador KNN
        # Cada combinación de parámetros es una configuración diferente
param_grid = {'n_neighbors': [1, 3, 5, 7, 9], 'weights': ['uniform', 'distance'], 'p': [1, 2, 1.5, 3]}

# Llamada la función GridSearchCV que nos crea todas las cominaciones del grid anterior
sss = model_selection.StratifiedShuffleSplit(n_splits=1, test_size=0.2)

clasificadores_ho = model_selection.GridSearchCV(knn, param_grid, scoring='accuracy',cv=sss)

# Entrenamiento de todos los clasificadores con todos los datos de entrenamiento
clasificadores_ho.fit(X_train,y_train)
# Se muestra la mejor configuración y su accuracy asociado
print(clasificadores_ho.best_params_)
print(clasificadores_ho.best_score_)

# Almacenamos el DataFrame con los resultados
diccionarioResultados_ho = clasificadores_ho.cv_results_
# Se muestra el accuracy obtenido para cada posible combinación de parámetros

print(round(np.max(diccionarioResultados_ho['mean_test_score']), 4))

# Se obtiene el rendimiento en entrenamiento y test por la mejor configuración
y_pred_train = clasificadores_ho.predict(X_train)
y_pred_test = clasificadores_ho.predict(X_test)

accTrain_ho = metrics.accuracy_score(y_train,y_pred_train)*100
accTest_ho = metrics.accuracy_score(y_test,y_pred_test)*100

{'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
0.9464285714285714
0.9464


In [17]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
Test.assertEquals(round(accTrain_ho, 2), 100.00, 'Valor de accuracy en train incorrecto')
Test.assertEquals(round(accTest_ho, 2), 90.14, 'Valor de accuracy en test incorrecto')

1 test passed.
1 test passed.


In [18]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
indice = np.argmax(diccionarioResultados_ho['mean_test_score'])
Test.assertEquals(round(np.max(diccionarioResultados_ho['mean_test_score']), 4), 0.9464, "Accuracy de la mejor configuración incorrecto")
Test.assertEquals(diccionarioResultados_ho['params'][indice], {'n_neighbors': 1, 'weights': 'uniform', 'p': 1}, "Mejor configuración incorrecta")

1 test passed.
1 test passed.


### Método validación cruzada de k particiones

El método de validación cruzada de k particiones, divide el conjunto de ejemplos original en k particiones manteniendo la distribución de las clases. Posteriormente, para realizar el aprendizaje se fusionan 4 de estas particiones para formar el conjunto de entrenamiento y la partición restante se utiliza como conjunto de test. Este proceso se realiza k veces utilizando una partición diferente como conjunto de test cada vez. En Scikit-learn, existe una función llamada [*StratifiedKFold*](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold) (también en la librería *model_selection*) que realiza la división del conjunto de ejemplos original en particiones y forma los k conjuntos de entrenamiento y de test. La llamada al constuctor y sus principales parámetros de entrada son:

    validacionCruzada = model_selection.StratifiedKFold(n_splits=numeroParticiones, random_state=semilla)

Los parámetros de entrada son:
* numeroParticiones: valor entero que determina el número de particiones a generar. Por defecto es 3.
* semilla: valor que determina la semilla para la generación de números aleatorios.

Si se quisiera realizar la validación de modelos de forma *manual*, para generar los iteradores de índices se debería ejecutar el método *split* utilizando el objeto generado por el constructor (variable validacionCruzada). Al método split se le deben pasar tanto los datos se entrada como los de salida.

    iteradorIndices = validacionCruzada.split(X, y)
    
Una vez realizada esta instrucción se puede realizar el bucle para realizar las k iteraciones:
  
    for train_index, val_index in iteradorIndices:

De esta forma el bucle for se realizará K veces y en cada iteración tendremos:

* En la variable train_index estarán los índices de los ejemplos a utilizar como conjunto de entrenamiento. El conjunto de entrenamiento estará formado por todas las filas del campo data cuyos índices estén en train_index y todas las columnas (para las clases lo mismo). Es decir, X[train_index, :] e y[train_index].
* En la variable val_index estarán los índices de los ejemplos a utilizar como conjunto de validación. El conjunto de validación estará formado por todas las filas del campo data cuyos índices estén en val_index y todas las columnas (para las clases lo mismo). Es decir, X[val_index, :] e y[val_index].

Ejercicio: realizar el proceso de validación cruzada para el dataset Ionosphere y mostrar para cada partición (iteración del bucle for) el accuracy en train y en validación. Probar 5 y 10 como valores de K. Utilizar como clasificador KNN con la configuración por defecto.

In [19]:
# Se fija la semilla de numpy para que la generación aleatoria siempre nos de los mismos números
np.random.seed(seed=12)

# Listas para almacenar los resultados de accuracy en train y validación
listaMediasTrain = []
listaMediasVal = []
# Para las dos validaciones cruzadas (de 5 y de 10 particiones)
for k in [5, 10]:
    # Llamada al constructor de la validación cruzada
    validacionCruzada = model_selection.StratifiedKFold(n_splits=k,shuffle=True)
    # Llamada a la función que nos da los iteradores con los índices de los ejemplos a utilizar en entrenamiento y en validación
    iteradorIndices = validacionCruzada.split(X, y)
    # Se crear las variables (arrays de numpy) para almacenar los k resultados de entrenamiento y de validación
    train = np.zeros(k)
    val = np.zeros(k)
    # Variable para almacenar el índice de la partición
    i = 0
    for train_index, val_index in iteradorIndices:
        # Entrenar KNN con los datos de train
        knn = neighbors.KNeighborsClassifier()
        knn.fit(X.values[train_index],y.values[train_index])
        # Prededecir los datos de train y calcular el porcentaje de acierto entre 0 y 100 (almacenarlo en la posición correspondiente)
        train_pred = knn.predict(X.values[train_index,:])
        train[i] = metrics.accuracy_score(y.values[train_index],train_pred)*100
        # Prededecir los datos de validación y calcular el porcentaje de acierto entre 0 y 100 (almacenarlo en la posición correspondiente) 
        val_pred = knn.predict(X.values[val_index,:])
        val[i] = metrics.accuracy_score(y.values[val_index],val_pred)*100
        i = i + 1
    
    # Cálculo y guardado de las medias de las k particiones tanto de entrenamiento como de validación
    mediaTrain = np.mean(train)
    listaMediasTrain.append(mediaTrain)
    print ('La media de las {}% particiones en train es: {}%'.format(k, mediaTrain))
    mediaVal = np.mean(val)
    listaMediasVal.append(mediaVal)
    print ('La media de las {}% particiones en validación es: {}%'.format(k, mediaVal))

La media de las 5% particiones en train es: 87.03685815963397%
La media de las 5% particiones en validación es: 83.77464788732395%
La media de las 10% particiones en train es: 87.27476391400441%
La media de las 10% particiones en validación es: 83.77777777777779%


In [20]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
Test.assertEquals(list(map(lambda x: round(x, 2), listaMediasTrain)), [86.07,86.11], 'Valores de accuracy en test incorrectos')
Test.assertEquals(list(map(lambda x: round(x, 2), listaMediasVal)), [83.21,83.57], 'Valores de accuracy en validación incorrectos')

1 test failed. Valores de accuracy en test incorrectos


Exception: Valores de accuracy en test incorrectos

El proceso anterior (utilizando un número de particiones específico) habría que repetirlo para todas las posibles configuraciones y seleccionar la que mejor resultado obtenga en validación. Para evitar toda esta programación, al igual que hemos hecho con Hold-out, podemos utilizar la clase *GridSearchCV* especificando el número de particiones a realizar.

Ejercicio: repetid el ejercicio de selección de la mejor configuración de KNN utilizando una validación cruzada de 10 particiones como método de validación de modelos.

In [55]:
# Se importan las librerías que vamos a utilizar en este ejercicio
from sklearn import neighbors

# Se fija la semilla de numpy para que la generación aleatoria siempre nos de los mismos números
np.random.seed(seed=12)

# Realizamos el proceso para KNN por lo que hay que llamar al constructor de dicho clasificador
knn = neighbors.KNeighborsClassifier()
# Se define el grid de parámetros a utilizar
    # Estos parámetros nos darán todas las posibles configuraciones del clasificador KNN
        # Cada combinación de parámetros es una configuración diferente
param_grid = {'n_neighbors': [1, 3, 5, 7, 9], 'weights': ['uniform', 'distance'], 'p': [1, 2, 1.5, 3]}

# Llamada la función GridSearchCV que nos crea todas las cominaciones del grid anterior
clasificadores_vc = model_selection.GridSearchCV(knn, param_grid, scoring='accuracy', cv=10)
# Entrenamiento de todos los clasificadores con todos los datos de entrenamiento
clasificadores_vc.fit(X_train,y_train)
# Se muestra la mejor configuración y su accuracy asociado
print (clasificadores_vc.best_params_)
print (clasificadores_vc.best_score_)

# Almacenamos el DataFrame con los resultados
diccionarioResultados_vc = clasificadores_vc.cv_results_

# Se obtiene el rendimiento en entrenamiento y test por la mejor configuración
y_pred_train = clasificadores_vc.predict(X_train)
y_pred_test = clasificadores_vc.predict(X_test)
accTrain_vc = metrics.accuracy_score(y_train,y_pred_train)*100
accTest_vc = metrics.accuracy_score(y_test,y_pred_test)*100

{'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
0.8964285714285714


In [40]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
Test.assertEquals(round(accTrain_vc, 2), 100.00, 'Valor de accuracy en train incorrecto')
Test.assertEquals(round(accTest_vc, 2), 90.14, 'Valor de accuracy en test incorrecto')

1 test passed.
1 test passed.


In [41]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
indice = np.argmax(diccionarioResultados_vc['mean_test_score'])
Test.assertEquals(round(np.max(diccionarioResultados_vc['mean_test_score']), 4), 0.8964, "Accuracy de la mejor configuración incorrecto")
Test.assertEquals(diccionarioResultados_vc['params'][indice], {'n_neighbors': 1, 'weights': 'uniform', 'p': 1}, "Mejor configuración incorrecta")

1 test passed.
1 test passed.


### Método Leave-One-Out

El tercer método de validación es el leave-one-out. Es como el método de validación cruzada de k particiones cuando k es igual al número de ejemplos del dataset. La función de scikit-learn que realiza este proceso es [*LeaveOneOut*](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html#sklearn.model_selection.LeaveOneOut). Su llamada y su uso es el siguiente:

    loo = model_selection.LeaveOneOut()
    
Al igual que para la validación cruzada, para generar los iteradores de índices, se debe ejecutar el método *split* utilizando el objeto generado por el constructor (variable loo). En este caso, solamente es necesario pasar como parámetro de entrada los valores de entrada de los ejemplos (X).

    iteradorIndices = loo.split(X)
    
Una vez realizada esta instrucción se puede realizar el bucle para realizar las k iteraciones del mismo modo que anteriormente. La diferencia es que en cada iteración, en la variable *val_index* solo habrá un índice puesto que se usa un solo ejemplo para evaluar el modelo aprendido. Por tanto, para obtener el resultado en validación se debe almacenar, en cada iteración del bucle for, la predicción del ejemplo de validación (en un vector de tantos elementos como ejemplos del dataset) y al finalizar el bucle for se debe comparar este vector con las clases reales del problema para obtener el accuracy rate.

Ejercicio: realizar el método de validación leave-one-out con el dataset Ionosphere utilizando KNN con la configuración por defecto y mostrar el accuracy obtenido en validación.

In [60]:
# Se fija la semilla de numpy para que la generación aleatoria siempre nos de los mismos números
np.random.seed(seed=12)

# Llamada al constructor del proceso leave one out
loo = model_selection.LeaveOneOut()
# Llamada a la función que nos da los iteradores con los índices de los ejemplos a utilizar en entrenamiento y en test
iteradorIndices = loo.split(X)

salida = []
# Para cada partición de ejemplos
for train_index,val_index in iteradorIndices:
    # Entrenar KNN con los datos de train
    knn = neighbors.KNeighborsClassifier()
    knn.fit(X.values[train_index,:],y.values[train_index])
    # Predecir la clase del ejemplo de validación y añadirla a la lista
    salida.append(knn.predict(X.values[val_index,:]))

# Cálculo del accuracy de las predicciones realizadas en validación (entre 0.0 y 100.0)
accVal = metrics.accuracy_score(y.values,salida)*100
print('El accuracy en validación es: {:.2f}%'.format(accVal))

El accuracy en validación es: 84.62%


In [57]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
Test.assertEquals(round(accVal, 2), 83.57, 'Accuracy incorrecto')

1 test failed. Accuracy incorrecto


Exception: Accuracy incorrecto

Ejercicio: repetid el ejercicio de selección de la mejor configuración de KNN utilizando leave-one-out como método de validación de modelos.

NOTA: esta celda puede tardar varios minutos en ejecutarse. Se recomienda usar todos los cores del ordenador (n_jobs=-1 en GridSearchCV).

In [48]:
# Se fija la semilla de numpy para que la generación aleatoria siempre nos de los mismos números
np.random.seed(seed=12)

# Realizamos el proceso para KNN por lo que hay que llamar al constructor de dicho clasificador
knn = neighbors.KNeighborsClassifier()
# Se define el grid de parámetros a utilizar
    # Estos parámetros nos darán todas las posibles configuraciones del clasificador KNN
        # Cada combinación de parámetros es una configuración diferente
param_grid = {'n_neighbors': [1, 3, 5, 7, 9], 'weights': ['uniform', 'distance'], 'p': [1, 2, 1.5, 3]}

# Llamada la función GridSearchCV que nos crea todas las cominaciones del grid anterior
clasificadores_loo = model_selection.GridSearchCV(knn, param_grid, scoring='accuracy')
# Entrenamiento de todos los clasificadores con todos los ejemplos de entrenamiento
clasificadores_loo.fit(X_train.values,y_train.values)
# Se muestra la mejor configuración y su accuracy asociado
print (clasificadores_loo.best_params_)
print (clasificadores_loo.best_score_)
# Almacenamos el DataFrame con los resultados
diccionarioResultados_loo = clasificadores_loo.cv_results_
# Se muestra el accuracy obtenido para cada posible combinación de parámetros
print(clasificadores_loo.score)
# Se obtiene el rendimiento en entrenamiento y test por la mejor configuración
y_pred_train = clasificadores_loo.predict(X_train)
y_pred_test = clasificadores_loo.predict(X_test)

accTrain_loo =  metrics.accuracy_score(y_train,y_pred_train)*100
accTest_loo = metrics.accuracy_score(y_test,y_pred_test)*100

{'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
0.9035714285714285
<bound method BaseSearchCV.score of GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [1, 3, 5, 7, 9], 'p': [1, 2, 1.5, 3],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)>


In [49]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
Test.assertEquals(round(accTrain_loo, 2), 100.00, 'Valor de accuracy en train incorrecto')
Test.assertEquals(round(accTest_loo, 2), 90.14, 'Valor de accuracy en test incorrecto')

1 test passed.
1 test passed.


In [50]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
indice = np.argmax(diccionarioResultados_loo['mean_test_score'])
Test.assertEquals(round(np.max(diccionarioResultados_loo['mean_test_score']),4), 0.9, "Accuracy de la mejor configuración incorrecto")
Test.assertEquals(diccionarioResultados_loo['params'][indice], {'n_neighbors': 1, 'weights': 'uniform', 'p': 1}, "Mejor configuración incorrecta")

1 test failed. Accuracy de la mejor configuración incorrecto


Exception: Accuracy de la mejor configuración incorrecto